#Homework 2 - Spark

## Lettura dei dati

In [0]:
import requests

from pyspark.sql.functions import when, count, isnull, expr, current_date, col, date_format, year, month, lag, sum, round
from pyspark.sql.window import Window

In [0]:
local_path_popolazione_regioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/popolazione_regioni.csv"
local_path_dati_covid_regioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_covid_regioni.csv"
local_path_dati_vaccini_somministrazioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_vaccini_somministrazioni.csv"
local_path_dati_vaccini_consegne = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_vaccini_consegne.csv"

# local_path_popolazione_regioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/popolazione_regioni.csv"
# local_path_dati_covid_regioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_covid_regioni.csv"
# local_path_dati_vaccini_somministrazioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_vaccini_somministrazioni.csv"
# local_path_dati_vaccini_consegne = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_vaccini_consegne.csv"

popolazione_regioni_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-statistici-riferimento/popolazione-istat-regione-range.csv"
dati_covid_regioni_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv"
dati_vaccini_somministrazioni_URL = "https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/somministrazioni-vaccini-latest.csv"
dati_vaccini_consegne_URL = "https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/consegne-vaccini-latest.csv"

dbutils.fs.put(local_path_popolazione_regioni, requests.get(popolazione_regioni_URL).text, True)
dbutils.fs.put(local_path_dati_covid_regioni, requests.get(dati_covid_regioni_URL).text, True)
dbutils.fs.put(local_path_dati_vaccini_somministrazioni, requests.get(dati_vaccini_somministrazioni_URL).text, True)
dbutils.fs.put(local_path_dati_vaccini_consegne, requests.get(dati_vaccini_consegne_URL).text, True)

popolazione_regioni = spark.read.option("inferSchema", "true") \
                                .option("header", "true") \
                                .csv(local_path_popolazione_regioni)
dati_covid_regioni = spark.read.option("inferSchema", "true") \
                                .option("header", "true") \
                                .csv(local_path_dati_covid_regioni)
dati_vaccini_somministrazioni = spark.read.option("inferSchema", "true") \
                                          .option("header", "true") \
                                          .csv(local_path_dati_vaccini_somministrazioni)
dati_vaccini_consegne = spark.read.option("inferSchema", "true") \
                                  .option("header", "true") \
                                  .csv(local_path_dati_vaccini_consegne)

Wrote 17300 bytes.
Wrote 1495697 bytes.
Wrote 3886362 bytes.
Wrote 57624 bytes.

## Preprocessing

### Popolazione Regioni
Si eliminano le colonne che non sono di interesse.

In [0]:
popolazione_regioni = popolazione_regioni.drop('codice_regione', 'latitudine_regione', 'longitudine_regione')
popolazione_regioni.show()

+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|range_eta|totale_genere_maschile|totale_genere_femminile|totale_generale|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
 ITF| Sud| ITF1| Abruzzo| ABR| 0-15| 87423| 82474| 169897|
 ITF| Sud| ITF1| Abruzzo| ABR| 16-19| 24402| 22349| 46751|
 ITF| Sud| ITF1| Abruzzo| ABR| 20-29| 67472| 62611| 130083|
 ITF| Sud| ITF1| Abruzzo| ABR| 30-39| 76472| 74241| 150713|
 ITF| Sud| ITF1| Abruzzo| ABR| 40-49| 94745| 95315| 190060|
 ITF| Sud| ITF1| Abruzzo| ABR| 50-59| 100048| 104109| 204157|
 ITF| Sud| ITF1| Abruzzo| ABR| 60-69| 80545| 87160| 167705|
 ITF| Sud| ITF1| Abruzzo| ABR| 70-79| 60810| 69762| 130572|
 ITF| Sud| ITF1| Abruzzo| ABR| 80-89| 34181| 50307| 84488|
 ITF| Sud| ITF1| Abruzzo| ABR| 90+| 5645| 13870| 19515|
 ITF| Sud| ITF5| Basilicata| BAS| 0-15| 36113| 33881| 69994|
 ITF| Sud| ITF5| Basilicata| BAS| 16-19| 11598| 10434| 22032|
 ITF| Sud| ITF5| Basilicata| BAS| 20-29| 32068| 29115| 61183|
 ITF| Sud| ITF5| Basilicata| BAS| 30-39| 33556| 31648| 65204|
 ITF| Sud| ITF5| Basilicata| BAS| 40-49| 39260| 39207| 78467|
 ITF| Sud| ITF5| Basilicata| BAS| 50-59| 43150| 44672| 87822|
 ITF| Sud| ITF5| Basilicata| BAS| 60-69| 35360| 37457| 72817|
 ITF| Sud| ITF5| Basilicata| BAS| 70-79| 24050| 27755| 51805|
 ITF| Sud| ITF5| Basilicata| BAS| 80-89| 14507| 21600| 36107|
 ITF| Sud| ITF5| Basilicata| BAS| 90+| 2488| 5335| 7823|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
only showing top 20 rows

Si verifica che il formato dei dati sia corretto.

In [0]:
popolazione_regioni.dtypes

Out[205]: [('codice_nuts_1', 'string'),
 ('descrizione_nuts_1', 'string'),
 ('codice_nuts_2', 'string'),
 ('denominazione_regione', 'string'),
 ('sigla_regione', 'string'),
 ('range_eta', 'string'),
 ('totale_genere_maschile', 'int'),
 ('totale_genere_femminile', 'int'),
 ('totale_generale', 'int')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = popolazione_regioni.groupBy('denominazione_regione') \
                .count() \
                .count()
print('Il numero di regioni è:', num_regioni)
print()
popolazione_regioni.groupBy('codice_nuts_1', 'descrizione_nuts_1', 'codice_nuts_2', 'denominazione_regione', 'sigla_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('descrizione_nuts_1') \
                    .show(num_regioni)

Il numero di regioni è: 21

+-------------+------------------+-------------+---------------------+-------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|
+-------------+------------------+-------------+---------------------+-------------+
 ITI| Centro| ITI4| Lazio| LAZ|
 ITI| Centro| ITI3| Marche| MAR|
 ITI| Centro| ITI2| Umbria| UMB|
 ITI| Centro| ITI1| Toscana| TOS|
 ITG| Isole| ITG1| Sicilia| SIC|
 ITG| Isole| ITG2| Sardegna| SAR|
 ITH| Nord-Est| ITH5| Emilia-Romagna| EMR|
 ITH| Nord-Est| ITH1| Bolzano| BOL|
 ITH| Nord-Est| ITH4| Friuli Venezia Gi...| FVG|
 ITH| Nord-Est| ITH2| Trento| TRE|
 ITH| Nord-Est| ITH3| Veneto| VEN|
 ITC| Nord-Ovest| ITC1| Piemonte| PIE|
 ITC| Nord-Ovest| ITC2| Valle d'Aosta| VDA|
 ITC| Nord-Ovest| ITC3| Liguria| LIG|
 ITC| Nord-Ovest| ITC4| Lombardia| LOM|
 ITF| Sud| ITF1| Abruzzo| ABR|
 ITF| Sud| ITF2| Molise| MOL|
 ITF| Sud| ITF3| Campania| CAM|
 ITF| Sud| ITF5| Basilicata| BAS|
 ITF| Sud| ITF4| Puglia| PUG|
 ITF| Sud| ITF6| Calabria| CAL|
+-------------+------------------+-------------+---------------------+-------------+

In [0]:
popolazione_regioni.groupBy('range_eta') \
                    .count() \
                    .orderBy('range_eta') \
                    .show()

+---------+-----+
range_eta|count|
+---------+-----+
 0-15| 21|
 16-19| 21|
 20-29| 21|
 30-39| 21|
 40-49| 21|
 50-59| 21|
 60-69| 21|
 70-79| 21|
 80-89| 21|
 90+| 21|
+---------+-----+

Si verifica la correttezza delle colonne numeriche.

In [0]:
popolazione_regioni.describe(['totale_genere_maschile', 'totale_genere_femminile', 'totale_generale']) \
                    .show()

+-------+----------------------+-----------------------+-----------------+
summary|totale_genere_maschile|totale_genere_femminile| totale_generale|
+-------+----------------------+-----------------------+-----------------+
 count| 210| 210| 210|
 mean| 138333.79047619048| 145673.29523809525|284007.0857142857|
 stddev| 151728.9786894938| 152578.20019731886|303588.4857882117|
 min| 457| 1307| 1764|
 max| 795710| 796399| 1592109|
+-------+----------------------+-----------------------+-----------------+

Si verifica la presenza di *missing values*.

In [0]:
popolazione_regioni.select([count(when(isnull(c), c)).alias(c) for c in popolazione_regioni.columns]).show()

+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|range_eta|totale_genere_maschile|totale_genere_femminile|totale_generale|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+

### Dati Covid Regioni
Si eliminano le colonne che non sono di interesse.

In [0]:
dati_covid_regioni = dati_covid_regioni.drop('stato', 'codice_regione', 'lat', 'long', 'note_test', 'note_casi', 'note',
                                             'casi_da_sospetto_diagnostico', 'casi_da_screening',
                                             'casi_testati', 'ingressi_terapia_intensiva', 
                                             'totale_positivi_test_molecolare', 'totale_positivi_test_antigenico_rapido',
                                             'tamponi_test_molecolare', 'tamponi_test_antigenico_rapido')
dati_covid_regioni.show()

+-------------------+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
 data|denominazione_regione|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|isolamento_domiciliare|totale_positivi|variazione_totale_positivi|nuovi_positivi|dimessi_guariti|deceduti|totale_casi|tamponi|codice_nuts_1|codice_nuts_2|
+-------------------+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
2020-02-24 18:00:00| Abruzzo| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 5| null| null|
2020-02-24 18:00:00| Basilicata| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| Calabria| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| Campania| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 10| null| null|
2020-02-24 18:00:00| Emilia-Romagna| 10| 2| 12| 6| 18| 0| 18| 0| 0| 18| 148| null| null|
2020-02-24 18:00:00| Friuli Venezia Gi...| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 58| null| null|
2020-02-24 18:00:00| Lazio| 1| 1| 2| 0| 2| 0| 2| 1| 0| 3| 124| null| null|
2020-02-24 18:00:00| Liguria| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| Lombardia| 76| 19| 95| 71| 166| 0| 166| 0| 6| 172| 1463| null| null|
2020-02-24 18:00:00| Marche| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 16| null| null|
2020-02-24 18:00:00| Molise| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| P.A. Bolzano| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| P.A. Trento| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 3| null| null|
2020-02-24 18:00:00| Piemonte| 2| 0| 2| 1| 3| 0| 3| 0| 0| 3| 141| null| null|
2020-02-24 18:00:00| Puglia| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| Sardegna| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| Sicilia| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 5| null| null|
2020-02-24 18:00:00| Toscana| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 140| null| null|
2020-02-24 18:00:00| Umbria| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| Valle d'Aosta| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 7| null| null|
+-------------------+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
only showing top 20 rows

Si verifica che il formato dei dati sia corretto.

In [0]:
dati_covid_regioni.dtypes

Out[211]: [('data', 'timestamp'),
 ('denominazione_regione', 'string'),
 ('ricoverati_con_sintomi', 'int'),
 ('terapia_intensiva', 'int'),
 ('totale_ospedalizzati', 'int'),
 ('isolamento_domiciliare', 'int'),
 ('totale_positivi', 'int'),
 ('variazione_totale_positivi', 'int'),
 ('nuovi_positivi', 'int'),
 ('dimessi_guariti', 'int'),
 ('deceduti', 'int'),
 ('totale_casi', 'int'),
 ('tamponi', 'int'),
 ('codice_nuts_1', 'string'),
 ('codice_nuts_2', 'string')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = dati_covid_regioni.groupBy('denominazione_regione') \
                .count() \
                .count()
print('Il numero di regioni è:', num_regioni)
print()
dati_covid_regioni.groupBy('codice_nuts_1', 'codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('codice_nuts_1') \
                    .show(50)

Il numero di regioni è: 21

+-------------+-------------+---------------------+
codice_nuts_1|codice_nuts_2|denominazione_regione|
+-------------+-------------+---------------------+
 null| null| Lazio|
 null| null| Sicilia|
 null| null| Emilia-Romagna|
 null| null| Toscana|
 null| null| Puglia|
 null| null| Abruzzo|
 null| null| Sardegna|
 null| null| Umbria|
 null| null| Piemonte|
 null| null| P.A. Bolzano|
 null| null| Molise|
 null| null| Calabria|
 null| null| Friuli Venezia Gi...|
 null| null| Valle d'Aosta|
 null| null| P.A. Trento|
 null| null| Marche|
 null| null| Basilicata|
 null| null| Veneto|
 null| null| Campania|
 null| null| Lombardia|
 null| null| Liguria|
 1| null| P.A. Bolzano|
 172410| ITI| Umbria|
 500340| ITH| P.A. Bolzano|
 ITC| ITC2| Valle d'Aosta|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF1| Abruzzo|
 ITF| ITF3| Campania|
 ITF| ITF4| Puglia|
 ITG| ITG1| Sicilia|
 ITG| ITG2| Sardegna|
 ITH| ITH3| Veneto|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH2| P.A. Trento|
 ITH| ITH1| P.A. Bolzano|
 ITH| ITH4| Friuli Venezia Gi...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+-------------+-------------+---------------------+

Si possono fare alcune osservazioni:
- per alcune entry mancano i codici nuts;
- per aclune entry i codici nuts sono sbagliati (problema dovuto alla lettura del file csv);
- alcune regioni (Bolzano e Trento) hanno una denominazione diversa rispetto alla tabella `popolazione_regioni`;

In [0]:
join1 = popolazione_regioni.join(dati_covid_regioni, on='denominazione_regione', how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .withColumnRenamed('denominazione_regione', 'popolazione_regioni_denominazione_regione')

join2 = dati_covid_regioni.join(popolazione_regioni, on='denominazione_regione', how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .withColumnRenamed('denominazione_regione', 'dati_covid_regioni_denominazione_regione')

join1.join(join2, on='codice_nuts_2').show()

dati_covid_regioni = dati_covid_regioni.replace('P.A. Trento', value='Trento', subset='denominazione_regione')
dati_covid_regioni = dati_covid_regioni.replace('P.A. Bolzano', value='Bolzano', subset='denominazione_regione')

+-------------+-----------------------------------------+----------------------------------------+
codice_nuts_2|popolazione_regioni_denominazione_regione|dati_covid_regioni_denominazione_regione|
+-------------+-----------------------------------------+----------------------------------------+
 ITH1| Bolzano| P.A. Bolzano|
 ITH2| Trento| P.A. Trento|
+-------------+-----------------------------------------+----------------------------------------+

In [0]:
dati_covid_regioni = dati_covid_regioni.drop('codice_nuts_1', 'codice_nuts_2')

data = popolazione_regioni.groupBy('codice_nuts_1', 'codice_nuts_2', 'denominazione_regione') \
                          .count() \
                          .drop('count')

dati_covid_regioni = dati_covid_regioni.join(data, on=['denominazione_regione'], how='left')

dati_covid_regioni.groupBy('codice_nuts_1', 'codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('codice_nuts_1').show(num_regioni)

+-------------+-------------+---------------------+
codice_nuts_1|codice_nuts_2|denominazione_regione|
+-------------+-------------+---------------------+
 ITC| ITC2| Valle d'Aosta|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF4| Puglia|
 ITF| ITF1| Abruzzo|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF3| Campania|
 ITG| ITG2| Sardegna|
 ITG| ITG1| Sicilia|
 ITH| ITH3| Veneto|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH1| Bolzano|
 ITH| ITH2| Trento|
 ITH| ITH4| Friuli Venezia Gi...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+-------------+-------------+---------------------+

Si verifica la correttezza delle colonne numeriche.

In [0]:
dati_covid_regioni.describe(['ricoverati_con_sintomi', 'terapia_intensiva', 'totale_ospedalizzati', 'dimessi_guariti']) \
                  .show()

+-------+----------------------+------------------+--------------------+-----------------+
summary|ricoverati_con_sintomi| terapia_intensiva|totale_ospedalizzati| dimessi_guariti|
+-------+----------------------+------------------+--------------------+-----------------+
 count| 9030| 9030| 9030| 9030|
 mean| 711.5446290143965| 81.5201550387597| 793.0647840531561|41435.85315614618|
 stddev| 1291.348304793182|143.02714647442633| 1429.6259353010903|85935.09667657697|
 min| 0| 0| 0| 0|
 max| 12077| 1381| 13328| 715042|
+-------+----------------------+------------------+--------------------+-----------------+

In [0]:
dati_covid_regioni.describe(['isolamento_domiciliare', 'totale_positivi', 'variazione_totale_positivi', 'nuovi_positivi', 'deceduti', 'totale_casi', 'tamponi']) \
                  .show()

+-------+----------------------+------------------+--------------------------+-----------------+------------------+------------------+-----------------+
summary|isolamento_domiciliare| totale_positivi|variazione_totale_positivi| nuovi_positivi| deceduti| totale_casi| tamponi|
+-------+----------------------+------------------+--------------------------+-----------------+------------------+------------------+-----------------+
 count| 9030| 9030| 9030| 9030| 9030| 9030| 9030|
 mean| 11636.879512735326|12429.944296788482| 49.008859357696565|441.7077519379845|2454.9551495016613|56320.754595791805|839681.7949058693|
 stddev| 21224.49630732643|22227.883873254505| 826.9902786331483|854.6881081976843| 4675.718255080165|105515.44475204292|1324027.249163621|
 min| 0| 0| -21926| -229| 0| 0| 0|
 max| 155066| 164406| 10263| 11489| 32789| 800100| 9370398|
+-------+----------------------+------------------+--------------------------+-----------------+------------------+------------------+-----------------+

In [0]:
dati_covid_regioni.filter(condition='nuovi_positivi < 0').show()

+---------------------+-------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
denominazione_regione| data|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|isolamento_domiciliare|totale_positivi|variazione_totale_positivi|nuovi_positivi|dimessi_guariti|deceduti|totale_casi|tamponi|codice_nuts_1|codice_nuts_2|
+---------------------+-------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
 Piemonte|2020-02-27 18:00:00| 2| 0| 2| 0| 2| -1| -1| 0| 0| 2| 156| ITC| ITC1|
 Liguria|2020-03-01 17:00:00| 12| 1| 13| 8| 21| -17| -17| 4| 0| 25| 121| ITC| ITC3|
 Liguria|2020-03-02 18:00:00| 12| 1| 13| 5| 18| -3| -3| 4| 0| 22| 121| ITC| ITC3|
 Sicilia|2020-03-02 18:00:00| 2| 0| 2| 3| 5| -2| -2| 2| 0| 7| 307| ITG| ITG1|
 Piemonte|2020-03-09 18:00:00| 222| 50| 272| 65| 337| -18| -10| 0| 13| 350| 1681| ITC| ITC1|
 Calabria|2020-04-17 17:00:00| 154| 7| 161| 658| 819| -28| -18| 99| 73| 991| 21657| ITF| ITF6|
 Sardegna|2020-05-04 17:00:00| 91| 9| 100| 553| 653| -36| -2| 545| 119| 1317| 28052| ITG| ITG2|
 Basilicata|2020-05-07 17:00:00| 48| 2| 50| 105| 155| -17| -16| 202| 26| 383| 16272| ITF| ITF5|
 Basilicata|2020-05-08 17:00:00| 48| 2| 50| 102| 152| -3| -1| 204| 26| 382| 16777| ITF| ITF5|
 Marche|2020-05-19 17:00:00| 144| 17| 161| 1967| 2128| -187| -3| 3561| 986| 6675| 89985| ITI| ITI3|
 Sardegna|2020-05-25 17:00:00| 49| 3| 52| 179| 231| -14| -2| 994| 129| 1354| 51073| ITG| ITG2|
 Sardegna|2020-06-09 17:00:00| 9| 1| 10| 44| 54| -2| -1| 1176| 131| 1361| 64272| ITG| ITG2|
 Campania|2020-06-12 17:00:00| 61| 2| 63| 283| 346| -264| -229| 3832| 430| 4608| 230551| ITF| ITF3|
+---------------------+-------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+

Si verifica la presenza di *missing values*.

In [0]:
dati_covid_regioni.select([count(when(isnull(c), c)).alias(c) for c in dati_covid_regioni.drop('data').columns]).show()

+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
denominazione_regione|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|isolamento_domiciliare|totale_positivi|variazione_totale_positivi|nuovi_positivi|dimessi_guariti|deceduti|totale_casi|tamponi|codice_nuts_1|codice_nuts_2|
+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+

### Dati Somministrazioni Vaccini

Si elmininano le colonne che non sono di interesse

In [0]:
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.drop('area', 'codice_regione_ISTAT')
dati_vaccini_somministrazioni.show()

+---------------------+---------------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+----------+
data_somministrazione| fornitore|fascia_anagrafica|sesso_maschile|sesso_femminile|categoria_operatori_sanitari_sociosanitari|categoria_personale_non_sanitario|categoria_ospiti_rsa|categoria_60_69|categoria_70_79|categoria_over80|categoria_forze_armate|categoria_personale_scolastico|categoria_soggetti_fragili|categoria_altro|prima_dose|seconda_dose|codice_NUTS1|codice_NUTS2| nome_area|
+---------------------+---------------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+----------+
 2020-12-27|Pfizer/BioNTech| 20-29| 1| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 30-39| 1| 3| 4| 0| 0| 0| 0| 0| 0| 0| 0| 0| 4| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 40-49| 1| 6| 7| 0| 0| 0| 0| 0| 0| 0| 0| 0| 7| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 50-59| 4| 5| 9| 0| 0| 0| 0| 0| 0| 0| 0| 0| 9| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 60-69| 10| 4| 13| 1| 0| 0| 0| 0| 0| 0| 0| 0| 14| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 70-79| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 80-89| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 20-29| 4| 5| 9| 0| 0| 0| 0| 0| 0| 0| 0| 0| 9| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 30-39| 10| 18| 27| 0| 0| 0| 0| 0| 0| 0| 0| 1| 28| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 40-49| 7| 23| 29| 0| 0| 0| 0| 0| 0| 0| 0| 1| 30| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 50-59| 11| 15| 26| 0| 0| 0| 0| 0| 0| 0| 0| 0| 26| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 60-69| 9| 4| 13| 0| 0| 0| 0| 0| 0| 0| 0| 0| 13| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 20-29| 15| 34| 49| 0| 0| 0| 0| 0| 0| 0| 0| 0| 49| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 30-39| 23| 20| 43| 0| 0| 0| 0| 0| 0| 0| 0| 0| 43| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 40-49| 22| 27| 49| 0| 0| 0| 0| 0| 0| 0| 0| 0| 49| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 50-59| 48| 28| 75| 1| 0| 0| 0| 0| 0| 0| 0| 0| 76| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 60-69| 43| 16| 59| 0| 0| 0| 0| 0| 0| 0| 0| 0| 59| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 20-29| 27| 33| 59| 1| 0| 0| 0| 0| 0| 0| 0| 0| 60| 0| ITF| ITF3| Campania|
 2020-12-27|Pfizer/BioNTech| 30-39| 61| 64| 122| 2| 0| 0| 0| 0| 0| 0| 0| 1| 125| 0| ITF| ITF3| Campania|
 2020-12-27|Pfizer/BioNTech| 40-49| 80| 87| 164| 3| 0| 0| 0| 0| 0| 0| 0| 0| 167| 0| ITF| ITF3| Campania|
+---------------------+---------------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+----------+
only showing top 20 rows

Si verifica che il formato dei dati sia corretto

In [0]:
dati_vaccini_somministrazioni.dtypes

Out[220]: [('data_somministrazione', 'string'),
 ('fornitore', 'string'),
 ('fascia_anagrafica', 'string'),
 ('sesso_maschile', 'int'),
 ('sesso_femminile', 'int'),
 ('categoria_operatori_sanitari_sociosanitari', 'int'),
 ('categoria_personale_non_sanitario', 'int'),
 ('categoria_ospiti_rsa', 'int'),
 ('categoria_60_69', 'int'),
 ('categoria_70_79', 'int'),
 ('categoria_over80', 'int'),
 ('categoria_forze_armate', 'int'),
 ('categoria_personale_scolastico', 'int'),
 ('categoria_soggetti_fragili', 'int'),
 ('categoria_altro', 'int'),
 ('prima_dose', 'int'),
 ('seconda_dose', 'int'),
 ('codice_NUTS1', 'string'),
 ('codice_NUTS2', 'string'),
 ('nome_area', 'string')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = dati_vaccini_somministrazioni.groupBy('nome_area') \
                                           .count() \
                                           .count()
print('Il numero di regioni è:', num_regioni)
print()
dati_vaccini_somministrazioni.groupBy('codice_NUTS1', 'codice_NUTS2', 'nome_area') \
                             .count() \
                             .drop('count') \
                             .orderBy('codice_NUTS1') \
                             .show(50)

Il numero di regioni è: 21

+------------+------------+--------------------+
codice_NUTS1|codice_NUTS2| nome_area|
+------------+------------+--------------------+
 ITC| ITC2|Valle d'Aosta / V...|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF4| Puglia|
 ITF| ITF1| Abruzzo|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF3| Campania|
 ITG| ITG2| Sardegna|
 ITG| ITG1| Sicilia|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH1|Provincia Autonom...|
 ITH| ITH4|Friuli-Venezia Gi...|
 ITH| ITH3| Veneto|
 ITH| ITH2|Provincia Autonom...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+------------+------------+--------------------+

In [0]:
join1 = popolazione_regioni.join(dati_vaccini_somministrazioni, popolazione_regioni.denominazione_regione == dati_vaccini_somministrazioni.nome_area, how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') 

join2 = dati_vaccini_somministrazioni.join(popolazione_regioni, popolazione_regioni.denominazione_regione == dati_vaccini_somministrazioni.nome_area, how='anti') \
                    .groupBy('codice_NUTS2', 'nome_area') \
                    .count() \
                    .drop('count') 

join1.join(join2, popolazione_regioni.codice_nuts_2 == dati_vaccini_somministrazioni.codice_NUTS2).show()

dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Provincia Autonoma Trento', value='Trento', subset='nome_area')
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Provincia Autonoma Bolzano / Bozen', value='Bolzano', subset='nome_area')
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Friuli-Venezia Giulia', value='Friuli Venezia Giulia', subset='nome_area')
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Valle d\'Aosta / Vallée d\'Aoste', value='Valle d\'Aosta', subset='nome_area')

+-------------+---------------------+------------+--------------------+
codice_nuts_2|denominazione_regione|codice_NUTS2| nome_area|
+-------------+---------------------+------------+--------------------+
 ITC2| Valle d'Aosta| ITC2|Valle d'Aosta / V...|
 ITH2| Trento| ITH2|Provincia Autonom...|
 ITH4| Friuli Venezia Gi...| ITH4|Friuli-Venezia Gi...|
 ITH1| Bolzano| ITH1|Provincia Autonom...|
+-------------+---------------------+------------+--------------------+

In [0]:
dati_vaccini_somministrazioni.groupBy('codice_NUTS1', 'codice_NUTS2', 'nome_area') \
                             .count() \
                             .drop('count') \
                             .orderBy('codice_NUTS1') \
                             .show(num_regioni)

+------------+------------+--------------------+
codice_NUTS1|codice_NUTS2| nome_area|
+------------+------------+--------------------+
 ITC| ITC2| Valle d'Aosta|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF4| Puglia|
 ITF| ITF1| Abruzzo|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF3| Campania|
 ITG| ITG2| Sardegna|
 ITG| ITG1| Sicilia|
 ITH| ITH3| Veneto|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH1| Bolzano|
 ITH| ITH2| Trento|
 ITH| ITH4|Friuli Venezia Gi...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+------------+------------+--------------------+

In [0]:
dati_vaccini_somministrazioni.groupBy('fascia_anagrafica') \
                    .count() \
                    .drop('count') \
                    .orderBy('fascia_anagrafica') \
                    .show()

+-----------------+
fascia_anagrafica|
+-----------------+
 16-19|
 20-29|
 30-39|
 40-49|
 50-59|
 60-69|
 70-79|
 80-89|
 90+|
+-----------------+

Si osserva che rispetto alla tabella popolazione_regioni manca la `fascia_anagrafica` 0-15

Si verifica la correttezza delle colonne numeriche.

In [0]:
dati_vaccini_somministrazioni.describe(['sesso_maschile', 'sesso_femminile', 'prima_dose', 'seconda_dose']) \
                             .show()

+-------+------------------+-----------------+-----------------+------------------+
summary| sesso_maschile| sesso_femminile| prima_dose| seconda_dose|
+-------+------------------+-----------------+-----------------+------------------+
 count| 41366| 41366| 41366| 41366|
 mean|192.08400618865736|257.9819658656868|318.4527147899241|131.61325726442004|
 stddev| 543.7704838041998|696.4389713540271|960.1204823995166| 517.6151019436899|
 min| 0| 0| 0| 0|
 max| 13943| 18424| 29486| 15529|
+-------+------------------+-----------------+-----------------+------------------+

In [0]:
dati_vaccini_somministrazioni.describe(['categoria_operatori_sanitari_sociosanitari', 'categoria_personale_non_sanitario', 'categoria_ospiti_rsa', 'categoria_60_69', 'categoria_70_79', 'categoria_over80', 'categoria_forze_armate', 'categoria_personale_scolastico', 'categoria_soggetti_fragili', 'categoria_altro']) \
                             .show()

+-------+------------------------------------------+---------------------------------+--------------------+------------------+-----------------+------------------+----------------------+------------------------------+--------------------------+------------------+
summary|categoria_operatori_sanitari_sociosanitari|categoria_personale_non_sanitario|categoria_ospiti_rsa| categoria_60_69| categoria_70_79| categoria_over80|categoria_forze_armate|categoria_personale_scolastico|categoria_soggetti_fragili| categoria_altro|
+-------+------------------------------------------+---------------------------------+--------------------+------------------+-----------------+------------------+----------------------+------------------------------+--------------------------+------------------+
 count| 41366| 41366| 41366| 41366| 41366| 41366| 41366| 41366| 41366| 41366|
 mean| 77.8090944253735| 21.240995020064787| 15.628728907798676| 19.75873906106464|64.50067688439782|142.99347290044963| 7.695619590968428| 27.8906106464246| 64.26171251752648| 8.286322100275589|
 stddev| 231.40832487809544| 67.11268994544935| 61.89608546842796|253.27350655293708|652.6725696871858| 898.5573363438707| 43.20981969391534| 154.01875852268367| 279.0388001140033|37.549599351846794|
 min| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 max| 4661| 2303| 2284| 20019| 28025| 31026| 1214| 3583| 5781| 2008|
+-------+------------------------------------------+---------------------------------+--------------------+------------------+-----------------+------------------+----------------------+------------------------------+--------------------------+------------------+

Si verifica la presenza di *missing values*.

In [0]:
dati_vaccini_somministrazioni.select([count(when(isnull(c), c)).alias(c) for c in dati_vaccini_somministrazioni.drop('data').columns]).show()

+---------------------+---------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+---------+
data_somministrazione|fornitore|fascia_anagrafica|sesso_maschile|sesso_femminile|categoria_operatori_sanitari_sociosanitari|categoria_personale_non_sanitario|categoria_ospiti_rsa|categoria_60_69|categoria_70_79|categoria_over80|categoria_forze_armate|categoria_personale_scolastico|categoria_soggetti_fragili|categoria_altro|prima_dose|seconda_dose|codice_NUTS1|codice_NUTS2|nome_area|
+---------------------+---------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+---------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---------------------+---------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+---------+

## Analisi

### Dati covid Regioni

#### Panoramica Casi per Mese

In [0]:
casi_mensili = dati_covid_regioni.select(date_format('data','yyyy-MM').alias('mese'), 'nuovi_positivi') \
                  .groupby('mese') \
                  .sum('nuovi_positivi') \
                  .withColumnRenamed('sum(nuovi_positivi)', 'casi')

In [0]:
tamponi_mensili = dati_covid_regioni.select(date_format('data','yyyy-MM').alias('mese'), 'denominazione_regione', 'tamponi') \
                                    .groupby('mese', 'denominazione_regione') \
                                    .max('tamponi') \
                                    .groupby('mese') \
                                    .sum('max(tamponi)') \
                                    .withColumnRenamed('sum(max(tamponi))', 'tamponi')

my_window = Window.partitionBy().orderBy("mese") 

tamponi_mensili = tamponi_mensili.withColumn("prev_value", lag(tamponi_mensili.tamponi) \
                                 .over(my_window)) 
tamponi_mensili = tamponi_mensili.withColumn("diff", when(isnull(tamponi_mensili.tamponi - tamponi_mensili.prev_value), tamponi_mensili.tamponi).otherwise(tamponi_mensili.tamponi - tamponi_mensili.prev_value)) \
                                 .drop('tamponi', 'prev_value') \
                                 .withColumnRenamed('diff', 'tamponi')

In [0]:
panoramica_mensile = casi_mensili.join(tamponi_mensili, on='mese') \
                                 .orderBy('mese', ascending=False)

panoramica_mensile = panoramica_mensile.withColumn('percentuale_tamponi_positivi', panoramica_mensile.casi / panoramica_mensile.tamponi * 100)

panoramica_mensile.show()

+-------+------+-------+----------------------------+
 mese| casi|tamponi|percentuale_tamponi_positivi|
+-------+------+-------+----------------------------+
2021-04|411680|8008409| 5.140596590408906|
2021-03|648200|9769610| 6.634860552263601|
2021-02|372503|7466161| 4.989217350121435|
2021-01|445585|5948709| 7.4904487679595695|
2020-12|508914|4772918| 10.662533904835575|
2020-11|922206|6160638| 14.969326228874348|
2020-10|364607|4450539| 8.192423434554781|
2020-09| 45623|2689063| 1.6966132812805055|
2020-08| 21702|1831746| 1.1847712510358968|
2020-07| 7006|1423003| 0.4923390885331935|
2020-06| 7584|1511371| 0.5017960513996894|
2020-05| 27556|1899522| 1.4506807502097896|
2020-04| 99671|1472249| 6.769982523336745|
2020-03|104664| 488307| 21.434056853577772|
2020-02| 1120| 18661| 6.001821981673007|
+-------+------+-------+----------------------------+

#### Attuali Positivi per Regione

In [0]:
attuali_positivi_regione = dati_covid_regioni.where(col("data") >= current_date() - expr("INTERVAL 1 days")) \
                                             .select('data','denominazione_regione', 'totale_positivi') \
                                             .withColumnRenamed('denominazione_regione', 'regione') \
                                             .withColumnRenamed('totale_positivi', 'attuali_positivi') \
                                             .withColumn('data', date_format(dati_covid_regioni.data, 'yyyy-MM-dd'))

popolazione = popolazione_regioni.groupBy('denominazione_regione') \
                                 .sum('totale_generale')

cond = [popolazione.denominazione_regione == attuali_positivi_regione.regione]
attuali_positivi_regione = attuali_positivi_regione.join(popolazione, cond, how='left') \
                                                   .drop('denominazione_regione') \
                                                   .withColumnRenamed('sum(totale_generale)', 'popolazione') \
                                                   .orderBy('regione')

attuali_positivi_regione = attuali_positivi_regione.withColumn('positivi_per_100k', (attuali_positivi_regione.attuali_positivi / attuali_positivi_regione.popolazione * 100000).cast('int')) \
                                                   .orderBy('positivi_per_100k', ascending=False)

attuali_positivi_regione.show(21)

+----------+--------------------+----------------+-----------+-----------------+
 data| regione|attuali_positivi|popolazione|positivi_per_100k|
+----------+--------------------+----------------+-----------+-----------------+
2021-04-28| Campania| 91803| 5712143| 1607|
2021-04-27| Campania| 91679| 5712143| 1604|
2021-04-27| Puglia| 48661| 3953305| 1230|
2021-04-28| Puglia| 48643| 3953305| 1230|
2021-04-28| Basilicata| 6130| 553254| 1107|
2021-04-27| Basilicata| 6059| 553254| 1095|
2021-04-27| Sardegna| 17548| 1611621| 1088|
2021-04-28| Sardegna| 17455| 1611621| 1083|
2021-04-27| Emilia-Romagna| 47803| 4464119| 1070|
2021-04-28| Emilia-Romagna| 45545| 4464119| 1020|
2021-04-27| Lazio| 46178| 5755700| 802|
2021-04-28| Lazio| 45645| 5755700| 793|
2021-04-28| Calabria| 14887| 1894110| 785|
2021-04-27| Calabria| 14675| 1894110| 774|
2021-04-28|Friuli Venezia Gi...| 8128| 1206216| 673|
2021-04-27|Friuli Venezia Gi...| 8044| 1206216| 666|
2021-04-27| Abruzzo| 8614| 1293941| 665|
2021-04-28| Abruzzo| 8556| 1293941| 661|
2021-04-27| Valle d'Aosta| 783| 125034| 626|
2021-04-28| Valle d'Aosta| 750| 125034| 599|
2021-04-27| Toscana| 21766| 3692555| 589|
+----------+--------------------+----------------+-----------+-----------------+
only showing top 21 rows

#### Dati Critici per Mese

In [0]:
decessi_mensili = dati_covid_regioni.select(date_format('data','yyyy-MM').alias('mese'), 'denominazione_regione', 'deceduti') \
                                    .groupby('mese', 'denominazione_regione') \
                                    .max('deceduti') \
                                    .groupby('mese') \
                                    .sum('max(deceduti)') \
                                    .withColumnRenamed('sum(max(deceduti))', 'deceduti')

my_window = Window.partitionBy().orderBy("mese") 

decessi_mensili = decessi_mensili.withColumn("prev_value", lag(decessi_mensili.deceduti) \
                                 .over(my_window)) 
decessi_mensili = decessi_mensili.withColumn("diff", when(isnull(decessi_mensili.deceduti - decessi_mensili.prev_value), decessi_mensili.deceduti).otherwise(decessi_mensili.deceduti - decessi_mensili.prev_value)) \
                                 .drop('deceduti', 'prev_value') \
                                 .withColumnRenamed('diff', 'deceduti')

In [0]:
terapie_intensive_mensili = dati_covid_regioni.select('data', 'terapia_intensiva') \
                  .groupby('data') \
                  .sum('terapia_intensiva') \
                  .select(date_format('data','yyyy-MM').alias('mese'), 'sum(terapia_intensiva)') \
                  .groupby('mese') \
                  .mean('sum(terapia_intensiva)') \
                  .withColumnRenamed('avg(sum(terapia_intensiva))', 'media_terapia_intensiva')

terapie_intensive_mensili = terapie_intensive_mensili.withColumn('media_terapia_intensiva', terapie_intensive_mensili.media_terapia_intensiva.cast('int'))

In [0]:
dati_critici_mensili = decessi_mensili.join(casi_mensili, on='mese') \
                                      .join(terapie_intensive_mensili, on='mese') \
                                      .orderBy('mese', ascending=False)

dati_critici_mensili.show()

+-------+--------+------+-----------------------+
 mese|deceduti| casi|media_terapia_intensiva|
+-------+--------+------+-----------------------+
2021-04| 10910|411680| 3356|
2021-03| 11647|648200| 3127|
2021-02| 9183|372503| 2128|
2021-01| 14357|445585| 2483|
2020-12| 18583|508914| 3005|
2020-11| 16958|922206| 3231|
2020-10| 2724|364607| 785|
2020-09| 411| 45623| 194|
2020-08| 341| 21702| 58|
2020-07| 311| 7006| 57|
2020-06| 1406| 7584| 207|
2020-05| 5458| 27556| 882|
2020-04| 15539| 99671| 2975|
2020-03| 12399|104664| 1985|
2020-02| 29| 1120| 53|
+-------+--------+------+-----------------------+

### Dati Somministrazioni Vaccini

#### Percentuale Popolazione Vaccinata per Regione
Si riporta la percentuale di popolazione vaccinata (prima e seconda dose) per regione.

In [0]:
dosi_somministrate_regione = dati_vaccini_somministrazioni.groupBy('nome_area') \
                                                          .sum('prima_dose', 'seconda_dose') \
                                                          .orderBy('nome_area') \
                                                          .withColumnRenamed('nome_area', 'regione') \
                                                          .withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                          .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')

popolazione = popolazione_regioni.groupBy('denominazione_regione').sum('totale_generale')

cond = [popolazione.denominazione_regione == dosi_somministrate_regione.regione]
dosi_somministrate_regione = dosi_somministrate_regione.join(popolazione, cond, how='left') \
                                                       .drop('denominazione_regione') \
                                                       .withColumnRenamed('sum(totale_generale)', 'popolazione') \
                                                       .orderBy('regione')

dosi_somministrate_regione = dosi_somministrate_regione.withColumn('percentuale_prima_dose', dosi_somministrate_regione.vaccinati_prima_dose / dosi_somministrate_regione.popolazione * 100)
dosi_somministrate_regione = dosi_somministrate_regione.withColumn('percentuale_seconda_dose', dosi_somministrate_regione.vaccinati_seconda_dose / dosi_somministrate_regione.popolazione * 100)

dosi_somministrate_regione.show(21)

+--------------------+--------------------+----------------------+-----------+----------------------+------------------------+
 regione|vaccinati_prima_dose|vaccinati_seconda_dose|popolazione|percentuale_prima_dose|percentuale_seconda_dose|
+--------------------+--------------------+----------------------+-----------+----------------------+------------------------+
 Abruzzo| 293298| 115704| 1293941| 22.66703041328778| 8.941984217209285|
 Basilicata| 118368| 52122| 553254| 21.394874686852695| 9.420989274365843|
 Bolzano| 131307| 50146| 532644| 24.651925113208822| 9.414543297211646|
 Calabria| 352974| 152021| 1894110| 18.635348527804616| 8.025985819197407|
 Campania| 1167766| 439155| 5712143| 20.443570827971218| 7.688095343551449|
 Emilia-Romagna| 1047099| 490642| 4464119| 23.455893536888244| 10.99079124010807|
Friuli Venezia Gi...| 272326| 126284| 1206216| 22.576885068677583| 10.46943499340085|
 Lazio| 1275281| 556259| 5755700| 22.156835832305365| 9.664489115138037|
 Liguria| 398066| 168036| 1524826| 26.10566713841448| 11.02001146360306|
 Lombardia| 2195405| 882980| 10027602| 21.893619232195295| 8.805495072500882|
 Marche| 381520| 133754| 1512672| 25.221594635188595| 8.842234139324322|
 Molise| 73176| 35474| 300516| 24.350117797388492| 11.804363162027979|
 Piemonte| 1019991| 424914| 4311217| 23.659003942506256| 9.856010495412317|
 Puglia| 918068| 296682| 3953305| 23.222797127972672| 7.504657495437361|
 Sardegna| 338135| 142940| 1611621| 20.98104951474323| 8.869330940711247|
 Sicilia| 925817| 420002| 4875290| 18.98998828787621| 8.61491316413998|
 Toscana| 836472| 339669| 3692555| 22.652932725443495| 9.19875262521479|
 Trento| 122803| 41633| 545425| 22.51510290140716| 7.633130127881927|
 Umbria| 205685| 81986| 870165| 23.63747105434027| 9.42189125050996|
 Valle d'Aosta| 31226| 10969| 125034| 24.97400707007694| 8.772813794647856|
 Veneto| 1068332| 482942| 4879133| 21.895939299051694| 9.898110996359394|
+--------------------+--------------------+----------------------+-----------+----------------------+------------------------+

#### Percentuale Popolazione Vaccinata per Fascia Anagrafica
Si riporta la percentuale di popolazione vaccinata (prima e seconda dose) per fascia anagrafica.

In [0]:
dosi_somministrate_fascia = dati_vaccini_somministrazioni.groupBy('fascia_anagrafica') \
                                                          .sum('prima_dose', 'seconda_dose') \
                                                          .orderBy('fascia_anagrafica') \
                                                          .withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                          .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')

popolazione = popolazione_regioni.groupBy('range_eta').sum('totale_generale')

cond = [popolazione.range_eta == dosi_somministrate_fascia.fascia_anagrafica]
dosi_somministrate_fascia = dosi_somministrate_fascia.join(popolazione, cond, how='left') \
                                                       .withColumnRenamed('sum(totale_generale)', 'popolazione') \
                                                       .drop('range_eta') \
                                                       .orderBy('fascia_anagrafica')

dosi_somministrate_fascia = dosi_somministrate_fascia.withColumn('percentuale_prima_dose', dosi_somministrate_fascia.vaccinati_prima_dose / dosi_somministrate_fascia.popolazione * 100)
dosi_somministrate_fascia = dosi_somministrate_fascia.withColumn('percentuale_seconda_dose', dosi_somministrate_fascia.vaccinati_seconda_dose / dosi_somministrate_fascia.popolazione * 100)

dosi_somministrate_fascia.show(9)

+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
fascia_anagrafica|vaccinati_prima_dose|vaccinati_seconda_dose|popolazione|percentuale_prima_dose|percentuale_seconda_dose|
+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
 16-19| 24535| 8185| 2298846| 1.0672746238765014| 0.356048208535935|
 20-29| 547638| 276542| 6084382| 9.000716917511097| 4.545112387749487|
 30-39| 795952| 368850| 6854632| 11.611885218637557| 5.381032854863689|
 40-49| 1160459| 472673| 8937229| 12.98455035671571| 5.288809316623754|
 50-59| 1591188| 643310| 9414195| 16.902008084599903| 6.833404236899702|
 60-69| 1963936| 521274| 7364364| 26.66810059904698| 7.078330185743127|
 70-79| 3277030| 442644| 5968373| 54.90658844546077| 7.416493573709284|
 80-89| 3119140| 2215348| 3628160| 85.97029899453167| 61.059820956076905|
 90+| 693237| 495488| 791543| 87.58045993711018| 62.59773632007358|
+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+

#### Percentuale Popolazione Vaccinata per Regione e Fascia Anagrafica
Si riporta la percentuale di popolazione vaccinata (prima e seconda dose) per regione e fascia anagrafica.

In [0]:
dosi_somministrate = dati_vaccini_somministrazioni.groupBy('nome_area', 'fascia_anagrafica') \
                                                  .sum('prima_dose', 'seconda_dose') \
                                                  .orderBy('nome_area', 'fascia_anagrafica') \
                                                  .withColumnRenamed('nome_area', 'regione') \
                                                  .withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                  .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')

cond = [popolazione_regioni.denominazione_regione == dosi_somministrate.regione, popolazione_regioni.range_eta == dosi_somministrate.fascia_anagrafica]
dosi_somministrate = dosi_somministrate.join(popolazione_regioni.select('denominazione_regione', 'range_eta', 'totale_generale'), cond, how='left') \
                                       .drop('denominazione_regione', 'range_eta') \
                                       .withColumnRenamed('totale_generale', 'popolazione') \
                                       .orderBy('regione', 'fascia_anagrafica')

dosi_somministrate = dosi_somministrate.withColumn('percentuale_prima_dose', dosi_somministrate.vaccinati_prima_dose / dosi_somministrate.popolazione * 100)
dosi_somministrate = dosi_somministrate.withColumn('percentuale_seconda_dose', dosi_somministrate.vaccinati_seconda_dose / dosi_somministrate.popolazione * 100)

dosi_somministrate.show(189)

+--------------------+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
 regione|fascia_anagrafica|vaccinati_prima_dose|vaccinati_seconda_dose|popolazione|percentuale_prima_dose|percentuale_seconda_dose|
+--------------------+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
 Abruzzo| 16-19| 622| 198| 46751| 1.3304528245385125| 0.42352035250582876|
 Abruzzo| 20-29| 11529| 4782| 130083| 8.862802979636077| 3.676114480754595|
 Abruzzo| 30-39| 18007| 7866| 150713| 11.947874436843538| 5.219191443339327|
 Abruzzo| 40-49| 27894| 9373| 190060| 14.6764179732716| 4.931600547195623|
 Abruzzo| 50-59| 40956| 11808| 204157| 20.06103146108142| 5.783784048550871|
 Abruzzo| 60-69| 52528| 10452| 167705| 31.321666020691097| 6.232372320443636|
 Abruzzo| 70-79| 58981| 5436| 130572| 45.171246515332534| 4.16322029225255|
 Abruzzo| 80-89| 67699| 53831| 84488| 80.12853896411325| 63.71437363885995|
 Abruzzo| 90+| 15082| 11958| 19515| 77.28414040481681| 61.27594158339738|
 Basilicata| 16-19| 352| 28| 22032| 1.597676107480029| 0.1270878721859114|
 Basilicata| 20-29| 3962| 1180| 61183| 6.475655002206496| 1.9286403085824495|
 Basilicata| 30-39| 6962| 2782| 65204| 10.677259063861113| 4.266609410465615|
 Basilicata| 40-49| 11747| 3833| 78467| 14.970624593778275| 4.884856054137408|
 Basilicata| 50-59| 17205| 4752| 87822| 19.590763134522103| 5.410944865751179|
 Basilicata| 60-69| 19736| 3849| 72817| 27.103560981638903| 5.28585357814796|
 Basilicata| 70-79| 19815| 1081| 51805| 38.24920374481228| 2.086671170736415|
 Basilicata| 80-89| 30994| 28023| 36107| 85.83931093693744| 77.61098955881131|
 Basilicata| 90+| 7595| 6594| 7823| 97.08551706506455| 84.28991435510673|
 Bolzano| 16-19| 338| 128| 23269| 1.452576389187331| 0.5500881000472733|
 Bolzano| 20-29| 5823| 2794| 61094| 9.53121419451992| 4.573280518545193|
 Bolzano| 30-39| 8146| 3477| 63127| 12.90414561122816| 5.5079443027547645|
 Bolzano| 40-49| 12285| 5223| 75664| 16.236255022203423| 6.9028864453372805|
 Bolzano| 50-59| 16244| 6633| 83539| 19.444810208405656| 7.94000406995535|
 Bolzano| 60-69| 30387| 3799| 57041| 53.272207710243514| 6.6601216668711976|
 Bolzano| 70-79| 30881| 4852| 46613| 66.2497586510201| 10.40911333748096|
 Bolzano| 80-89| 22141| 18753| 27108| 81.67699572082043| 69.17884019477644|
 Bolzano| 90+| 5062| 4487| 6165| 82.10867802108677| 72.78183292781833|
 Calabria| 16-19| 714| 255| 76743| 0.9303780149329581| 0.33227786247605645|
 Calabria| 20-29| 12658| 5683| 213889| 5.9180228997283635| 2.6569856327347363|
 Calabria| 30-39| 22142| 11165| 236668| 9.355721939594707| 4.717579055892643|
 Calabria| 40-49| 32809| 13845| 266417| 12.314904829646757| 5.196740448244669|
 Calabria| 50-59| 48059| 19380| 286804| 16.756739794424067| 6.757227932664817|
 Calabria| 60-69| 74693| 22256| 241215| 30.965321393777334| 9.22662355160334|
 Calabria| 70-79| 71168| 14220| 175208| 40.61914981051093| 8.11606775946304|
 Calabria| 80-89| 75073| 54300| 107720| 69.69272187151874| 50.408466394355735|
 Calabria| 90+| 15658| 10917| 23058| 67.90701708734495| 47.34582357533178|
 Campania| 16-19| 3208| 976| 261694| 1.2258592096112253| 0.37295467225079676|
 Campania| 20-29| 50960| 23128| 693479| 7.348456117632978| 3.335068545694967|
 Campania| 30-39| 77792| 34307| 715258| 10.876075486048391| 4.796451070802424|
 Campania| 40-49| 112818| 40237| 835597| 13.501484567321329| 4.815359557298554|
 Campania| 50-59| 167584| 56272| 868684| 19.29171021913607| 6.477844647766045|
 Campania| 60-69| 251290| 62233| 670867| 37.457499027378006| 9.276503390388854|
 Campania| 70-79| 274642| 43549| 484380| 56.69969858375655| 8.990668483422107|
 Campania| 80-89| 195113| 155546| 255273| 76.4330736113886| 60.93319700869265|
 Campania| 90+| 34359| 22907| 49044| 70.05749938830438| 46.70703857760378|
 Emilia-Romagna| 16-19| 2002| 697| 160045| 1.250898184885501| 0.43550251491767944|
 Emilia-Romagna| 20-29| 43895| 2390

#### Categorie

In [0]:
dosi_somministrate_categorie_mensili = dati_vaccini_somministrazioni.select(date_format('data_somministrazione','yyyy-MM').alias('mese'), \
                                                                            'categoria_operatori_sanitari_sociosanitari', 'categoria_personale_non_sanitario', 'categoria_ospiti_rsa', \
                                                                            'categoria_60_69', 'categoria_70_79', 'categoria_over80', 'categoria_forze_armate', 'categoria_personale_scolastico', \
                                                                            'categoria_soggetti_fragili', 'categoria_altro') \
                                                                    .groupby('mese') \
                                                                    .agg(sum('categoria_operatori_sanitari_sociosanitari').alias('operatori_sanitari_sociosanitari'), \
                                                                         sum('categoria_personale_non_sanitario').alias('personale_non_sanitario'), \
                                                                         sum('categoria_ospiti_rsa').alias('ospiti_rsa'), \
                                                                         sum('categoria_60_69').alias('60_69'), \
                                                                         sum('categoria_70_79').alias('70_79'), \
                                                                         sum('categoria_over80').alias('over80'), \
                                                                         sum('categoria_forze_armate').alias('forze_armate'), \
                                                                         sum('categoria_personale_scolastico').alias('personale_scolastico'), \
                                                                         sum('categoria_soggetti_fragili').alias('soggetti_fragili'), \
                                                                         sum('categoria_altro').alias('altro')) \
                                                                      .orderBy('mese')
                                                                   
dosi_somministrate_categorie_mensili.show()

+-------+--------------------------------+-----------------------+----------+------+-------+-------+------------+--------------------+----------------+------+
 mese|operatori_sanitari_sociosanitari|personale_non_sanitario|ospiti_rsa| 60_69| 70_79| over80|forze_armate|personale_scolastico|soggetti_fragili| altro|
+-------+--------------------------------+-----------------------+----------+------+-------+-------+------------+--------------------+----------------+------+
2020-12| 34165| 1493| 3262| 163| 42| 51| 0| 8| 1| 909|
2021-01| 1470793| 229904| 201912| 11945| 4478| 17792| 176| 322| 138| 56778|
2021-02| 900427| 269791| 212285| 14415| 8354| 639808| 67403| 176815| 2612| 76358|
2021-03| 597769| 280556| 154285| 86822| 400645|2822964| 214900| 847257| 438548|116763|
2021-04| 215497| 96911| 74754|703995|2254616|2434453| 35858| 129321| 2216951| 91964|
+-------+--------------------------------+-----------------------+----------+------+-------+-------+------------+--------------------+----------------+------+

In [0]:
dosi_somministrate_mensili = dati_vaccini_somministrazioni.select(date_format('data_somministrazione','yyyy-MM').alias('mese'), 'prima_dose', 'seconda_dose') \
                                                          .groupby('mese') \
                                                          .sum('prima_dose', 'seconda_dose')

dosi_somministrate_mensili = dosi_somministrate_mensili.withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                       .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')


totale_dosi_somministrate_mensili = dosi_somministrate_mensili.withColumn('totale_dosi', dosi_somministrate_mensili.vaccinati_prima_dose + dosi_somministrate_mensili.vaccinati_seconda_dose)

percentuale_dosi_categorie = totale_dosi_somministrate_mensili.join(dosi_somministrate_categorie_mensili, on='mese') \
                                                              .drop('vaccinati_prima_dose', 'vaccinati_seconda_dose') \
                                                              .withColumn('operatori_sanitari_sociosanitari', round(dosi_somministrate_categorie_mensili.operatori_sanitari_sociosanitari / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('personale_non_sanitario', round(dosi_somministrate_categorie_mensili.personale_non_sanitario / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('ospiti_rsa', round(dosi_somministrate_categorie_mensili.ospiti_rsa / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('60_69', round(dosi_somministrate_categorie_mensili['60_69'] / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('70_79', round(dosi_somministrate_categorie_mensili['70_79'] / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('over80', round(dosi_somministrate_categorie_mensili.over80 / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('forze_armate', round(dosi_somministrate_categorie_mensili.forze_armate / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('personale_scolastico', round(dosi_somministrate_categorie_mensili.personale_scolastico / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('soggetti_fragili', round(dosi_somministrate_categorie_mensili.soggetti_fragili / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('altro', round(dosi_somministrate_categorie_mensili.altro / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .drop('totale_dosi') \
                                                              .orderBy('mese')
    
percentuale_dosi_categorie.show()

+-------+--------------------------------+-----------------------+----------+-----+------+------+------------+--------------------+----------------+-----+
 mese|operatori_sanitari_sociosanitari|personale_non_sanitario|ospiti_rsa|60_69| 70_79|over80|forze_armate|personale_scolastico|soggetti_fragili|altro|
+-------+--------------------------------+-----------------------+----------+-----+------+------+------------+--------------------+----------------+-----+
2020-12| 85.212| 3.724| 8.136|0.407| 0.105| 0.127| 0.0| 0.02| 0.002|2.267|
2021-01| 73.752| 11.528| 10.125|0.599| 0.225| 0.892| 0.009| 0.016| 0.007|2.847|
2021-02| 38.02| 11.392| 8.964|0.609| 0.353|27.016| 2.846| 7.466| 0.11|3.224|
2021-03| 10.029| 4.707| 2.588|1.457| 6.722|47.361| 3.605| 14.215| 7.358|1.959|
2021-04| 2.611| 1.174| 0.906|8.529|27.314|29.493| 0.434| 1.567| 26.858|1.114|
+-------+--------------------------------+-----------------------+----------+-----+------+------+------------+--------------------+----------------+-----+

#### Fornitori e Fasce Anagrafiche

In [0]:
dosi_fascia_fornitore = dati_vaccini_somministrazioni.groupBy('fascia_anagrafica', 'fornitore') \
                                                          .sum('prima_dose', 'seconda_dose') \
                                                          .orderBy('fascia_anagrafica', 'fornitore') 

totale_dosi_somministrate_fascia = dosi_somministrate_fascia.select('fascia_anagrafica', 'vaccinati_prima_dose', 'vaccinati_seconda_dose') \
                                                            .withColumn('totale_dosi', dosi_somministrate_fascia.vaccinati_prima_dose + dosi_somministrate_fascia.vaccinati_seconda_dose) \
                                                            .drop('vaccinati_prima_dose', 'vaccinati_seconda_dose')

dosi_fascia_fornitore = dosi_fascia_fornitore.join(totale_dosi_somministrate_fascia, on='fascia_anagrafica')

dosi_fascia_fornitore = dosi_fascia_fornitore.withColumn('percentuale_dosi', round((dosi_fascia_fornitore['sum(prima_dose)'] + dosi_fascia_fornitore['sum(seconda_dose)']) *100 /  dosi_fascia_fornitore.totale_dosi,3)) \
                                             .drop('sum(prima_dose)', 'sum(seconda_dose)', 'totale_dosi') \
                                             .orderBy('fascia_anagrafica', 'fornitore')

dosi_fascia_fornitore.show(100)

+-----------------+--------------------+----------------+
fascia_anagrafica| fornitore|percentuale_dosi|
+-----------------+--------------------+----------------+
 16-19| Janssen| 0.015|
 16-19| Moderna| 7.323|
 16-19| Pfizer/BioNTech| 85.092|
 16-19|Vaxzevria (AstraZ...| 7.57|
 20-29| Janssen| 0.058|
 20-29| Moderna| 4.223|
 20-29| Pfizer/BioNTech| 74.704|
 20-29|Vaxzevria (AstraZ...| 21.015|
 30-39| Janssen| 0.056|
 30-39| Moderna| 4.22|
 30-39| Pfizer/BioNTech| 70.032|
 30-39|Vaxzevria (AstraZ...| 25.691|
 40-49| Janssen| 0.038|
 40-49| Moderna| 4.527|
 40-49| Pfizer/BioNTech| 66.491|
 40-49|Vaxzevria (AstraZ...| 28.945|
 50-59| Janssen| 0.029|
 50-59| Moderna| 4.942|
 50-59| Pfizer/BioNTech| 68.777|
 50-59|Vaxzevria (AstraZ...| 26.252|
 60-69| Janssen| 0.338|
 60-69| Moderna| 5.941|
 60-69| Pfizer/BioNTech| 61.961|
 60-69|Vaxzevria (AstraZ...| 31.76|
 70-79| Janssen| 0.535|
 70-79| Moderna| 6.859|
 70-79| Pfizer/BioNTech| 47.792|
 70-79|Vaxzevria (AstraZ...| 44.814|
 80-89| Janssen| 0.007|
 80-89| Moderna| 9.448|
 80-89| Pfizer/BioNTech| 89.376|
 80-89|Vaxzevria (AstraZ...| 1.169|
 90+| Janssen| 0.003|
 90+| Moderna| 13.416|
 90+| Pfizer/BioNTech| 85.887|
 90+|Vaxzevria (AstraZ...| 0.695|
+-----------------+--------------------+----------------+

### Correlazione Dati

#### Dosi Somministrate e Media terapie intensive

In [0]:
dosi_somministrate_terapie_intensive = dosi_somministrate_mensili.join(terapie_intensive_mensili, on='mese', how='left') \
                                                                 .orderBy('mese', ascending=False)


dosi_somministrate_terapie_intensive.show()

+-------+--------------------+----------------------+-----------------------+
 mese|vaccinati_prima_dose|vaccinati_seconda_dose|media_terapia_intensiva|
+-------+--------------------+----------------------+-----------------------+
2021-04| 6045037| 2209283| 3356|
2021-03| 4145717| 1814792| 3127|
2021-02| 1588137| 780131| 2128|
2021-01| 1354130| 640108| 2483|
2020-12| 40094| 0| 3005|
+-------+--------------------+----------------------+-----------------------+

#### Dosi Somministrate e Media terapie intensive

In [0]:
attuali_terapie_intensive_regioni = dati_covid_regioni.where(col("data") >= current_date() - expr("INTERVAL 1 days")) \
                                             .select('data', 'denominazione_regione', 'terapia_intensiva') \
                                             .withColumnRenamed('denominazione_regione', 'regione') \
                                             .withColumn('data', date_format(dati_covid_regioni.data, 'yyyy-MM-dd'))

In [0]:
attuali_dosi_somministrate_terapie_intensive = attuali_terapie_intensive_regioni.join(dosi_somministrate_regione, on='regione') \
                                                                                .select('data', 'regione', 'percentuale_prima_dose', 'percentuale_seconda_dose', 'terapia_intensiva') \
                                                                                .orderBy('regione')

attuali_dosi_somministrate_terapie_intensive.show(21)

+----------+--------------------+----------------------+------------------------+-----------------+
 data| regione|percentuale_prima_dose|percentuale_seconda_dose|terapia_intensiva|
+----------+--------------------+----------------------+------------------------+-----------------+
2021-04-28| Abruzzo| 22.66703041328778| 8.941984217209285| 42|
2021-04-27| Abruzzo| 22.66703041328778| 8.941984217209285| 45|
2021-04-28| Basilicata| 21.394874686852695| 9.420989274365843| 9|
2021-04-27| Basilicata| 21.394874686852695| 9.420989274365843| 10|
2021-04-28| Bolzano| 24.651925113208822| 9.414543297211646| 4|
2021-04-27| Bolzano| 24.651925113208822| 9.414543297211646| 6|
2021-04-27| Calabria| 18.635348527804616| 8.025985819197407| 45|
2021-04-28| Calabria| 18.635348527804616| 8.025985819197407| 45|
2021-04-28| Campania| 20.443570827971218| 7.688095343551449| 136|
2021-04-27| Campania| 20.443570827971218| 7.688095343551449| 136|
2021-04-27| Emilia-Romagna| 23.455893536888244| 10.99079124010807| 246|
2021-04-28| Emilia-Romagna| 23.455893536888244| 10.99079124010807| 250|
2021-04-28|Friuli Venezia Gi...| 22.576885068677583| 10.46943499340085| 32|
2021-04-27|Friuli Venezia Gi...| 22.576885068677583| 10.46943499340085| 32|
2021-04-28| Lazio| 22.156835832305365| 9.664489115138037| 307|
2021-04-27| Lazio| 22.156835832305365| 9.664489115138037| 317|
2021-04-27| Liguria| 26.10566713841448| 11.02001146360306| 66|
2021-04-28| Liguria| 26.10566713841448| 11.02001146360306| 66|
2021-04-28| Lombardia| 21.893619232195295| 8.805495072500882| 575|
2021-04-27| Lombardia| 21.893619232195295| 8.805495072500882| 582|
2021-04-28| Marche| 25.221594635188595| 8.842234139324322| 68|
+----------+--------------------+----------------------+------------------------+-----------------+
only showing top 21 rows

# IDEE
* Dubito che metteremo mano, ma ci sono somministrazioni di vaccini a singola dose.